In [12]:
import pandas as pd
import json
import os

# Read the Excel file into a DataFrame
excel_file = '../inputdata/mapping.xlsx'
df_sources1 = pd.read_excel(excel_file, sheet_name='Sources1')
df_target1 = pd.read_excel(excel_file, sheet_name='Target1')
df_sources2 = pd.read_excel(excel_file, sheet_name='Sources2')
df_target2 = pd.read_excel(excel_file, sheet_name='Target2')

# Convert the table DataFrame to a list of dictionaries
json_data_sources1 = df_sources1.to_json(orient='records')
json_data_target1 = df_target1.to_json(orient='records')
json_data_sources2 = df_sources2.to_json(orient='records')
json_data_target2 = df_target2.to_json(orient='records')


prompt = f'''
You are an accurate sql mapping assistant 
write a big query sql that uses the sources and maps to the target definition.
The Jsons contain the metadata of the sources and targets.
Important : Dont Use the Json in the SQL.
The sql should be accurate and should not contain any errors.
The sql should be modular but no unnecessary repitition.

<EXAMPLE>
Json for the Sources = {json_data_sources1}  
Json for the Targets = {json_data_target1}
Query :
-- CTE for Latest Department
WITH LatestDepartment AS (
  SELECT ee.ID AS EmployeeID,
         ed.DEPID AS DepartmentID,
         d.name AS DepartmentName,
         ROW_NUMBER() OVER (PARTITION BY ee.ID ORDER BY ed.ValidFrom DESC) AS rn
  FROM EE ee
  LEFT JOIN ED ed ON ee.ID = ed.EMPID
  LEFT JOIN Dep d ON ed.DEPID = d.ID
),

-- CTE for Latest Department Address
LatestDepartmentAddress AS (
  SELECT ld.EmployeeID,
         ld.DepartmentID,
         ld.DepartmentName,
         a.City AS DepartmentAddressCity,
         a.State AS DepartmentAddressState,
         a.Zip AS DepartmentAddressZip,
         ROW_NUMBER() OVER (PARTITION BY ld.EmployeeID, ld.DepartmentID ORDER BY a.ValidFrom DESC) AS rn
  FROM LatestDepartment ld
  LEFT JOIN AD a ON ld.DepartmentID = a.DEPID
  WHERE ld.rn = 1
),
-- Final Query
SELECT ld.EmployeeID,
       CONCAT(CAST(ee.firstname AS VARCHAR(100)), ' ', CAST(ee.lastname AS VARCHAR(100))) AS EmployeeName,
       lda.DepartmentName,
       CAST(lda.DepartmentID AS VARCHAR(20)) AS DeptID,
       lda.DepartmentAddressCity,
       lda.DepartmentAddressState,
       lda.DepartmentAddressZip
FROM LatestDepartment ld
INNER JOIN EE ee ON ld.EmployeeID = ee.ID
LEFT JOIN LatestDepartmentAddress lda ON ld.EmployeeID = lda.EmployeeID AND ld.DepartmentID = lda.DepartmentID
WHERE ld.rn = 1;
<EXAMPLE>

Json for the Sources = {json_data_sources2}  
Json for the Targets = {json_data_target2}

'''

print(prompt)



You are an accurate sql mapping assistant 
write a big query sql that uses the sources and maps to the target definition.
The Jsons contain the metadata of the sources and targets.
Important : Dont Use the Json in the SQL.
The sql should be accurate and should not contain any errors.
The sql should be modular but no unnecessary repitition.

<EXAMPLE>
Json for the Sources = [{"SourceTableName":"EE","SourceColumnName ":"ID","SourceColumnDataType":"int","UniqueDataValues ":"1,2","TableDescription":"Employee table","Nullable":"N"},{"SourceTableName":"EE","SourceColumnName ":"firstname","SourceColumnDataType":"varchar","UniqueDataValues ":"Raj,Kamal","TableDescription":"Employee table","Nullable":"Y"},{"SourceTableName":"EE","SourceColumnName ":"lastname","SourceColumnDataType":"varchar","UniqueDataValues ":"Kumar,Hassan","TableDescription":"Employee table","Nullable":"Y"},{"SourceTableName":"Dep","SourceColumnName ":"ID","SourceColumnDataType":"int","UniqueDataValues ":"1,2","TableDescrip

In [13]:
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()

# Access the API key
api_key = os.getenv('GEMINI_API_KEY')

import os
import requests


# Set the API endpoint and request body
url = 'https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent'
payload = {
    "contents": [
        {
            "parts": [
                {
                    "text": prompt
                }
            ]
        }
    ]
}

# Make the API request
headers = {
    'Content-Type': 'application/json'
}
response = requests.post(url, headers=headers, json=payload, params={'key': api_key})

# Check the response
if response.status_code == 200:
    #print(response.json())
    print('')
else:
    print(f'Error: {response.status_code} - {response.text}')


data = json.loads(response.text)
#print(data)


# Extract the SQL code text
sql_code = data['candidates'][0]['content']['parts'][0]['text']

# Remove the leading and trailing triple backticks
sql_code = sql_code.strip('```')

print(sql_code)



WITH LatestInsurance AS (
  SELECT ptntid AS PatientID,
         insid AS InsuranceID,
         name AS InsuranceName,
         ROW_NUMBER() OVER (PARTITION BY ptntid ORDER BY ValidFrom DESC) AS rn
  FROM PTINS101
),

LatestInsuranceAddress AS (
  SELECT li.PatientID,
         li.InsuranceID,
         li.InsuranceName,
         a."Address Line 1" AS InsuranceAddressLine1,
         a.City AS InsuranceCity,
         a.State AS InsuranceState,
         a.Zip AS InsuranceZip,
         ROW_NUMBER() OVER (PARTITION BY li.PatientID, li.InsuranceID ORDER BY a.ValidFrom DESC,a.AddType = 'H') AS rn
  FROM LatestInsurance li
  LEFT JOIN ADD101 a ON li.InsuranceID = a.INSID
),

LatestPatientAddress AS (
  SELECT p.ID AS PatientID,
         a."Address Line 1" AS AddressLine1,
         a.City AS City,
         a.State AS State,
         a.Zip AS Zip,
         ROW_NUMBER() OVER (PARTITION BY p.ID ORDER BY a.ValidFrom DESC,a.AddType = 'H') AS rn
  FROM Ptnt101 p
  LEFT JOIN ADD101 a ON p.ID = a.ptnti